# Word2Vec from Scratch
(by Tevfik Aytekin)

In [1]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.corpus import gutenberg, brown
import gensim 
from gensim.models import Word2Vec 
from gensim.parsing.preprocessing import remove_stopwords
from nltk.tokenize import RegexpTokenizer
import numpy as np
from sklearn.utils import shuffle
from queue import PriorityQueue


# You need to call nltk.download() to download all the nltk corpora

## Definition from Wikipedia:

“Word2vec takes as its input a large corpus of text and produces a vector space, typically of several hundred dimensions, with each unique word in the corpus being assigned a corresponding vector in the space. Word vectors are positioned in the vector space such that words that share common contexts in the corpus are located in close proximity to one another in the space.”

In [2]:
nltk.download('brown')
num_sents = len(brown.sents())
print("number of sentences:", num_sents)

[nltk_data] Downloading package brown to
[nltk_data]     /Users/tevfikaytekin/nltk_data...
[nltk_data]   Package brown is already up-to-date!


number of sentences: 57340


An example sentence represented as a list of words

In [3]:
brown.sents()[0]

['The',
 'Fulton',
 'County',
 'Grand',
 'Jury',
 'said',
 'Friday',
 'an',
 'investigation',
 'of',
 "Atlanta's",
 'recent',
 'primary',
 'election',
 'produced',
 '``',
 'no',
 'evidence',
 "''",
 'that',
 'any',
 'irregularities',
 'took',
 'place',
 '.']

Following is an example application of word2vec using Gensim library. You can see some of the parameters and can find all the details of Gensim implementation [here](https://radimrehurek.com/gensim/models/word2vec.html). The Gensim word2vec source code is [here](https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/models/word2vec.py) and the original source code by Mikolov can be found [here](https://github.com/tmikolov/word2vec/blob/master/word2vec.c).



In [4]:
model = gensim.models.Word2Vec(brown.sents(),min_count = 5,
                              size = 30, window = 5, iter=5, negative=5) 

An example vector representation of the word "book". Since we set size = 30, the representation is an array of 30 reals.

In [5]:
model.wv['book']

array([ 0.26579976, -0.04866867, -1.7459654 , -0.6115054 , -0.5542139 ,
        0.8283093 ,  0.41846105, -0.04969763, -0.8726043 , -0.07534609,
       -0.37399644,  0.34195694, -0.07170147,  0.50888443,  0.52798414,
       -0.55636686,  0.7330005 ,  0.02543076,  0.7189497 ,  0.6839749 ,
       -0.49252346,  0.45718732,  0.54439783,  0.12184591,  0.2821921 ,
       -0.32804507, -1.1608193 ,  0.7788689 , -0.4628607 , -0.26199132],
      dtype=float32)

One way to test the performance of word2vec is to look at most similar words to a given word. Below you will find most similar words of the words "book" and "eight"

In [6]:
model.wv.most_similar(positive='book')

[('story', 0.9440566897392273),
 ('opinion', 0.9329782724380493),
 ('artist', 0.9126719236373901),
 ('novel', 0.9116226434707642),
 ('gapt', 0.9088972806930542),
 ('hero', 0.9082541465759277),
 ('statement', 0.9060463309288025),
 ('era', 0.9050542712211609),
 ('poem', 0.902087390422821),
 ('name', 0.9006975889205933)]

In [7]:
model.wv.most_similar(positive='eight')

[('seven', 0.9629417061805725),
 ('thirty', 0.943234920501709),
 ('fifteen', 0.9374372959136963),
 ('nine', 0.9368208050727844),
 ('65', 0.9360512495040894),
 ('eleven', 0.9357398152351379),
 ('flights', 0.9344480037689209),
 ('decades', 0.9337546825408936),
 ('twelve', 0.9332236051559448),
 ('previous', 0.9326519966125488)]

As you can see the results are quite amazing. But it might not be so for every word, for example for the word "angry" the results are not very satisying. However, if we have used a larger text the results could be better.

In [13]:
model.wv.most_similar(positive='on')

[('through', 0.851377546787262),
 ('into', 0.8492832183837891),
 ('over', 0.8413790464401245),
 ('toward', 0.8371174335479736),
 ('along', 0.8329604864120483),
 ('from', 0.8304653167724609),
 ('behind', 0.8277620077133179),
 ('against', 0.8031615614891052),
 ('across', 0.7904695272445679),
 ('around', 0.781238317489624)]

In [8]:
model.wv.most_similar(positive='angry')

[('verses', 0.9653607606887817),
 ('pathetic', 0.9590141177177429),
 ('dressed', 0.9582985639572144),
 ('guilt', 0.9556961059570312),
 ('impartial', 0.9556828737258911),
 ('imagery', 0.9555773735046387),
 ('master', 0.9553168416023254),
 ('complement', 0.9551525115966797),
 ('ankle', 0.9546823501586914),
 ('minister', 0.9534153938293457)]

You can also find (cosine) similarity between two words.

In [14]:
print("Cosine similarity between 'book' and 'story':", 
    model.wv.similarity('book', 'story')) 

Cosine similarity between 'book' and 'story': 0.9440565


In [15]:
print("Cosine similarity between 'book' and 'eight':", 
    model.wv.similarity('book', 'eight')) 

Cosine similarity between 'book' and 'eight': 0.5177929


## word2vec from scratch

Now we will write word2vec from scratch. Note that the purpose of this implementation is to help understand the theory behind word2vec. The implementation is not meant to be efficient so the running time is quite slow compared to the Gensim implementation. However, the code is simpler and shows the main ingredients of word2vec.

Different objectives can be used for word2vec. The following is the objective for word2vec with negative sampling. The main idea behind this objective is to find paramater values which maximizes the dot product of word representations which are in the same context and minimizes the dot product of word representations which are not in the same context.


$$
J(\Theta) = \underset{\theta}{\operatorname{argmax}}{\sum_{c,t \in D_p}log(\sigma(v_c \cdot v_t))+\sum_{c,t \in D_n}log(\sigma(-v_c \cdot v_t))}
$$

Here, $D_p$ is the set of word pairs whose distance is at most $m$ and $D_n$ is the set of unrelated (negative) word pairs, i.e., word pairs whose distance is larger than $m$, and $\sigma$ is the sigmoid function. Below we find the derivative of this function with respect to positive and negative words which we will use in the updates of gradient descent algorithm.
$$
\frac{\partial J(\Theta)}{\partial v_{c}}=\sum_{c,t \in D_p}\frac{1}{\sigma(v_c \cdot v_t)}\sigma(v_c \cdot v_t)(1-\sigma(v_c \cdot v_t))(v_t)\\
+ \sum_{c,t \in D_n}\frac{1}{\sigma(-v_c \cdot v_t)}\sigma(-v_c \cdot v_t)(1-\sigma(-v_c \cdot v_t))(-v_t)\\
= \sum_{c,t \in D_p}(1-\sigma(v_c \cdot v_t))v_t + \sum_{c,t \in D_n}-(1-\sigma(-v_c \cdot v_t))v_t 
$$
$$
\frac{\partial J(\Theta)}{\partial v_{t \in D_p}}=\sum_{c,t \in D_p}(1-\sigma(v_c \cdot v_t))v_c 
$$
$$
\frac{\partial J(\Theta)}{\partial v_{t \in D_n}}=\sum_{c,t \in D_n}-(1-\sigma(-v_c \cdot v_t))v_c 
$$

In [17]:
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))  

In [18]:
def build_indices(sents):
    """ 
  
    Parameters: 
    sents: A list of sentecens and each sentence is a list of words (i.e., a list of lists). 
  
    Returns: 
    word_freqs: frequency of each word
    word_to_index: a mapping from word names to integers.
    index_to_word: a mapping from integers to word names.
    
  
    """
    counter = 0
    word_freqs = {}
    word_to_index = {}
    index_to_word = {}
    for i in range(len(sents)): 
        for j in range(len(sents[i])):
            w = sents[i][j].lower()
            if w in word_freqs:
                word_freqs[w] += 1
            else:
                word_freqs[w] = 1
                word_to_index[w] = counter
                index_to_word[counter] = w
                counter += 1
            
    return word_freqs, word_to_index, index_to_word

In [19]:
def build_training_set(sents, word_freqs, window=5, sampling_freq = 0.001, neg_exp = 0.75, num_negs = 1, min_count=5):
    """ 
    Builds a trainig set
    
    Parameters: 
    sents: A list of sentecens and each sentence is a list of words (i.e., a list of lists).
    word_freqs: Frequency of words.
    windows: size of the context window.
    sampling_freq: words whose frequency larger than this value will be discarded.
    neg_exp: used for adjusting the negative sampling distribution.
    min_count: 
  
    Returns: 
    training_set: list of context word, positive and negatives
    """
    words_list = []
    total_freq = sum(word_freqs.values())
    
    #total_freq = sum([freq**(neg_exp) for freq in word_freqs.values()])
    word_array = []
    for word, freq in word_freqs.items():
        if ((word_freqs[word]/total_freq) < sampling_freq) and (word_freqs[word] > min_count):
            words_list.append(word)
            for i in range(int(freq**neg_exp)):
                word_array.append(word)
    
    training_set = []
    
    sampled_sents = []
    for i in range(len(sents)): 
        sent = []
        for j in range(len(sents[i])):
            w = sents[i][j].lower()
            if ((word_freqs[w] / total_freq) < sampling_freq) and (word_freqs[w] > min_count):
                sent.append(w)
        sampled_sents.append(sent)
    
    
    for i in range(len(sampled_sents)): 
        for j, w in enumerate(sampled_sents[i]):
            context = []
            for k in range(max(j-window,0),min(j+window+1,len(sampled_sents[i]))):
                w_p = sampled_sents[i][k]
                if (w == w_p):
                    continue
                w_n = []
                for k in range(num_negs):
                    w_n.append(word_array[np.random.randint(0,len(word_array))] )
                training_set.append([w,w_p,w_n])

    return training_set, np.unique(words_list)

In order to understand the produced training_set here is a very simple example sentence consisting of 6 words.

In [20]:
sents = [["a","b","c","d","e","f"]]
word_freqs = {"a":1,"b":1,"c":1,"d":1,"e":1,"f":1}


In [27]:
training_set, words_list = build_training_set(sents,window=1, word_freqs= word_freqs , sampling_freq = 1, min_count= 0, num_negs = 2)

In [28]:
# print training set
training_set

[['a', 'b', ['c', 'e']],
 ['b', 'a', ['b', 'e']],
 ['b', 'c', ['e', 'a']],
 ['c', 'b', ['a', 'e']],
 ['c', 'd', ['d', 'b']],
 ['d', 'c', ['c', 'f']],
 ['d', 'e', ['c', 'a']],
 ['e', 'd', ['f', 'b']],
 ['e', 'f', ['f', 'c']],
 ['f', 'e', ['c', 'b']]]

Let us now build the training set for the brown dataset which will take some time

In [29]:
word_freqs, word_to_index, index_to_word = build_indices(brown.sents())

In [30]:
training_set, words_list = build_training_set(brown.sents(),word_freqs)

In [31]:
# print first 10 examples in the trainigng set
training_set[:10]

[['fulton', 'county', ['separate']],
 ['fulton', 'grand', ['hanover']],
 ['fulton', 'jury', ['korean']],
 ['fulton', 'friday', ['critical']],
 ['fulton', 'investigation', ['miniature']],
 ['county', 'fulton', ['measures']],
 ['county', 'grand', ['l.']],
 ['county', 'jury', ['taxable']],
 ['county', 'friday', ['mine']],
 ['county', 'investigation', ['physically']]]

In [32]:
len(training_set)

3255198

In [34]:
def build_model(training_set, initial_alpha = 0.025, min_alpha = 0.0001, n_iters = 5, my_lambda = 0, vector_size = 30):
    word_vectors = {}
    
    # initialize word vectors
    for n in range(len(words_list)):
        word_vectors[words_list[n]] = np.random.rand(vector_size,1) - 0.5
    

    alpha = initial_alpha
    for t in range(n_iters):
        training_set = shuffle(training_set)
        objective = 0
        print("cosine of words 'friend' and 'fellow': ",np.dot(word_vectors['friend'].T, word_vectors['fellow']))
        for ex in training_set:
            w = ex[0]
            w_p = ex[1]
            w_n_list = ex[2]
            w_v = word_vectors[w]
            w_p_v = word_vectors[w_p]
            word_vectors[w_p] = w_p_v + alpha*(((1-sigmoid(np.dot(w_v.T,w_p_v)))*w_v)-my_lambda*w_p_v)
            objective += np.log((sigmoid(np.dot(w_v.T,w_p_v))))

            for n in range(len(w_n_list)):
                w_n = w_n_list[n]
                w_n_v = word_vectors[w_n]
                word_vectors[w_n] = w_n_v + alpha*((-(1-sigmoid(-np.dot(w_v.T,w_n_v)))*w_v)-my_lambda*w_n_v)      
                objective += np.log((sigmoid(-np.dot(w_v.T,w_p_v))))
     
        alpha = initial_alpha - ((initial_alpha - min_alpha) * t / n_iters)
        print("alpha: ",alpha)
        print("Iteration: ", t)
        print("Objective: ", objective)
    print("cosine of words 'friend' and 'fellow': ",np.dot(word_vectors['friend'].T, word_vectors['fellow']))

    return word_vectors

In [35]:
word_vectors = build_model(training_set, initial_alpha = 0.025, min_alpha = 0.0001, n_iters = 5, my_lambda = 0, vector_size = 30)

cosine of words 'friend' and 'fellow':  [[-0.28009892]]
alpha:  0.025
Iteration:  0
Objective:  [[-4629741.23324028]]
cosine of words 'friend' and 'fellow':  [[-0.17475462]]
alpha:  0.020020000000000003
Iteration:  1
Objective:  [[-4762469.56499669]]
cosine of words 'friend' and 'fellow':  [[0.37146372]]
alpha:  0.015040000000000001
Iteration:  2
Objective:  [[-4901301.54166369]]
cosine of words 'friend' and 'fellow':  [[0.63088071]]
alpha:  0.010060000000000001
Iteration:  3
Objective:  [[-4987241.78843278]]
cosine of words 'friend' and 'fellow':  [[0.89783896]]
alpha:  0.005080000000000001
Iteration:  4
Objective:  [[-5038862.9510509]]


In [36]:
print("cosine of words 'friend' and 'fellow': ",np.dot(word_vectors['friend'].T, word_vectors['fellow']))


cosine of words 'friend' and 'fellow':  [[0.95898103]]


In [44]:
def most_similar(word, word_vectors):
    pq = PriorityQueue()
    for w in word_vectors.keys():
        pq.put((-np.dot(word_vectors[word].T, word_vectors[w]), w))
    return pq

In [45]:
pq = most_similar('book', word_vectors)

In [46]:
for i in range(10):
    print(pq.get())

(array([[-3.16413106]]), 'book')
(array([[-2.37720672]]), 'james')
(array([[-2.28934352]]), 'david')
(array([[-2.08540662]]), 'rayburn')
(array([[-2.06554837]]), 'wrote')
(array([[-2.03319124]]), 'poet')
(array([[-2.03146274]]), 'poems')
(array([[-2.02708295]]), 'athletics')
(array([[-1.96002009]]), 'history')
(array([[-1.9494699]]), 'soul')


In [40]:
pq = most_similar('eight', word_vectors)
for i in range(10):
    print(pq.get())

(array([[-3.63661662]]), 'eight')
(array([[-2.94130014]]), 'hundred')
(array([[-2.77708548]]), '31')
(array([[-2.76095622]]), 'year')
(array([[-2.62741732]]), 'nine')
(array([[-2.52538252]]), '20')
(array([[-2.45696699]]), 'cent')
(array([[-2.36731842]]), 'thousand')
(array([[-2.33627123]]), 'four')
(array([[-2.32009323]]), 'years')


### Question
- Even though "opinion" and "story" does not appear in the context of "book" (window size 5) still word2vec puts these close to book, which is remarkable. Can you find this by building a co-occurrence matrix?
